In [83]:
from geopy.distance import great_circle
from geopy.geocoders import Nominatim
from geopy import Photon
import numpy as np
import random

In [84]:
def get_location(address):
    geolocator = Photon(user_agent="measurements")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)


def calculate_distance(start_coords, end_coords):
    return great_circle(start_coords, end_coords).kilometers


def calculate_path_distance(distance_matrix, city_sequence):
    total_distance = 0
    for i in range(len(city_sequence) - 1):
        start_city = city_sequence[i]
        next_city = city_sequence[i + 1]
        total_distance += distance_matrix[start_city][next_city]
    return total_distance

In [85]:
def print_generation_data(generation_number, entities):
    print(f"Generation {generation_number}")
    for entity in entities:
        print(entity, calculate_path_distance(distance_matrix, entity))


def generate_initial_entities(num_sequences, num_cities, start_city):
    sequences = []
    for _ in range(num_sequences):
        city_sequence = [i for i in range(num_cities) if i != start_city]
        random.shuffle(city_sequence)
        city_sequence.insert(0, start_city)
        city_sequence.append(start_city)
        sequences.append(city_sequence)
    return sequences

In [86]:
def select_entities(entities, selection_rate, distance_matrix):
    total_distance = [calculate_path_distance(distance_matrix, entity) for entity in entities]
    total_distance_sum = sum(total_distance)
    selection_probabilities = [d / total_distance_sum for d in total_distance]
    
    num_selected = int(len(entities) * selection_rate)
    selected_entities = random.choices(entities, weights=selection_probabilities, k=num_selected)
    return selected_entities


def create_offspring(parent1, parent2):
    num_cities = len(parent1)
    
    # Генерируем случайную битовую маску для выбора городов от родителя 1
    mask = [random.choice([True, False]) for _ in range(num_cities)]
    
    # Создаем потомка, выбирая города от родителей с учетом маски
    offspring = []
    for i in range(num_cities):
        if mask[i]:
            offspring.append(parent1[i])
        else:
            offspring.append(parent2[i])
    
    # Проверяем наличие повторяющихся городов и заменяем их
    offspring = fix_duplicates(offspring, num_cities)
    
    return offspring

def fix_duplicates(route, num_cities):
    # Находим все уникальные города в маршруте
    unique_cities = set(route)

    # Находим недостающие города
    missing_cities = list(set(range(num_cities - 1)) - unique_cities)

    # Заменяем дубликаты недостающими городами
    for i in range(1, len(route) - 1):
        if route[i] in unique_cities:
            unique_cities.remove(route[i])
        else:
            route[i] = missing_cities.pop()

    return route


def create_next_generation(entities, offspring_per_pair):
    next_generation = []
    num_entities = len(entities)
    
    # Скрещиваем пары сущностей и создаем потомков
    for i in range(0, num_entities, 2):
        parent1 = entities[i]
        parent2 = entities[(i + 1) % num_entities]  # Циклическая перестановка для последней пары
        
        for _ in range(offspring_per_pair):
            offspring = create_offspring(parent1, parent2)
            next_generation.append(offspring)
    
    return next_generation


def mutate_entity(entity, mutation_rate):
    if random.random() < mutation_rate:
        num_cities = len(entity)
        index1, index2 = random.sample(range(1, num_cities - 1), 2)
        entity[index1], entity[index2] = entity[index2], entity[index1]
    return entity

def apply_mutation(entities, mutation_rate):
    mutated_entities = [mutate_entity(entity, mutation_rate) for entity in entities]
    return mutated_entities


def genetic_algorithm(distance_matrix, 
                      start_city,
                      initial_entities, 
                      iterations, 
                      selection_rate, 
                      offspring_rate, 
                      mutation_rate, 
                      newcomers_rate):
    print_generation_data(0, initial_entities)
    entities = initial_entities
    for iteration in range(iterations):
        selected_entities = select_entities(entities, selection_rate, distance_matrix)
        entities = create_next_generation(selected_entities, offspring_rate)
        entities = apply_mutation(entities, mutation_rate)
        entities += generate_initial_entities(int(len(entities) * newcomers_rate), len(distance_matrix), start_city)
        print_generation_data(iteration+1, entities)
    shortest_distance = float('inf')
    shortest_path = None
    for entity in entities:
        distance = calculate_path_distance(distance_matrix, entity)
        if distance < shortest_distance:
            shortest_distance = distance
            shortest_path = entity

    return shortest_path

In [87]:
locations = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Елабуга']

coords = {location: get_location(location) for location in locations}
distance_matrix = np.zeros((len(locations), len(locations)))

for i, start in enumerate(locations):
    for j, end in enumerate(locations):
        if i != j:
            distance_matrix[i][j] = calculate_distance(coords[start], coords[end])

distance_matrix.round(2)

array([[   0.  ,  634.58, 2815.26, 1417.03,  718.98,  401.77,  901.72],
       [ 634.58,    0.  , 3110.05, 1782.25, 1199.49,  896.13, 1360.23],
       [2815.26, 3110.05,    0.  , 1401.76, 2117.87, 2415.63, 1938.51],
       [1417.03, 1782.25, 1401.76,    0.  ,  717.11, 1015.84,  540.92],
       [ 718.98, 1199.49, 2117.87,  717.11,    0.  ,  323.48,  183.26],
       [ 401.77,  896.13, 2415.63, 1015.84,  323.48,    0.  ,  503.62],
       [ 901.72, 1360.23, 1938.51,  540.92,  183.26,  503.62,    0.  ]])

In [88]:
start_city = 0
initial_entities_count = 6

evolution_iterations = 10

selection_rate = 0.6
offspring_rate = 4
mutation_rate = 0.2
newcomers_rate = 0.2

In [89]:
initial_entities = generate_initial_entities(initial_entities_count, len(distance_matrix), start_city)

path = genetic_algorithm(distance_matrix, start_city, initial_entities, evolution_iterations, selection_rate, offspring_rate, mutation_rate, newcomers_rate)

print("Best path:")
print(path, calculate_path_distance(distance_matrix, path))

Generation 0
[0, 6, 2, 5, 4, 1, 3, 0] 9978.113360059244
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 1, 4, 5, 6, 3, 2, 0] 7419.098091640093
[0, 2, 1, 3, 4, 6, 5, 0] 9513.316549617532
[0, 6, 3, 1, 5, 4, 2, 0] 9377.63175885518
[0, 4, 3, 1, 5, 2, 6, 0] 9370.337398824504
Generation 1
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 1, 6, 5, 4, 3, 2, 0] 7756.030917785936
[0, 1, 4, 5, 6, 3, 2, 0] 7419.098091640093
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 1, 4, 2, 0] 9490.320389615294
Generation 2
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 1, 2, 0] 9909.851016544217
[0, 3, 6, 5, 4, 